In [ ]:
%%javascript
$('#header').toggle()

In [ ]:
import ipywidgets as widgets
import json
# import networkx as nx
import pandas as pd
import qgrid
import requests
from ipyleaflet import FullScreenControl, LayerGroup, Map, Marker
from IPython.display import display, HTML, IFrame
from pyvis.network import Network

In [ ]:
layout_main = {
    'width': '100%',
    'height': '100%',
}

In [ ]:
image_header = widgets.Image(
    value=open('images/DI_NLP_Header.png', 'rb').read(),
    format='png',
    layout=layout_main,
)

In [ ]:
input_url = widgets.Text(
    placeholder='TextCat URL',
    layout=layout_main,
)
input_text = widgets.Textarea(
    placeholder='Text for analysis',
    layout=layout_main,
)

In [ ]:
output_message = widgets.Output()
output_text = widgets.Output()
output_tree = widgets.Output()
output_graph_raw = widgets.Output()
output_graph_con = widgets.Output()
output_map = widgets.Output()
output_table = widgets.Output(
    layout={
        'width': '3000px',
        'height': '100%',
    },
)

In [ ]:
accordion_text = widgets.Accordion(
    children=[output_text],
    layout=layout_main,
)
accordion_text.set_title(0, 'Text')

accordion_tree = widgets.Accordion(
    children=[output_tree],
    layout=layout_main,
)
accordion_tree.set_title(0, 'Tree')

accordion_graph_raw = widgets.Accordion(
    children=[output_graph_raw],
    layout=layout_main,
)
accordion_graph_raw.set_title(0, 'Graph Raw')

accordion_graph_con = widgets.Accordion(
    children=[output_graph_con],
    layout=layout_main,
)
accordion_graph_con.set_title(0, 'Graph Condensed')

accordion_map = widgets.Accordion(
    children=[output_map],
    layout=layout_main,
)
accordion_map.set_title(0, 'Map')

accordion_table = widgets.Accordion(
    children=[output_table],
    layout=layout_main,
)
accordion_table.set_title(0, 'Table')

In [ ]:
@output_message.capture(clear_output=True, wait=True)
def set_output_message(message):
    display(HTML(message))

@output_text.capture(clear_output=True, wait=True)
def set_output_text(html_ent):
    accordion_text.selected_index = 0
    display(HTML(html_ent))

@output_tree.capture(clear_output=True, wait=True)
def set_output_tree(html_dep):
    accordion_tree.selected_index = 0
    display(HTML(html_dep))

@output_graph_raw.capture(clear_output=True, wait=True)
def set_output_graph_raw(tokens, ents):
# g_nx is not currently used for anything. Started working on it here to use NetworkX graph methods if needed:
#     g_nx = nx.DiGraph()
#     g_nx.add_nodes_from([(token['idx_tok'], token) for token in tokens])
#     g_nx.add_edges_from([(token['idx_tok_head'], token['idx_tok'], {'dep': token['dep']}) for token in tokens])

    g_pyvis = Network(
        directed=True,
        notebook=True,
        width='100%',
        height='100%',
    )
    g_pyvis.prep_notebook()

#     g_pyvis.enable_physics(True)
#     g_pyvis.barnes_hut()
#     g_pyvis.show_buttons(filter_=['nodes'])
#     g_pyvis.show_buttons(filter_=['edges'])
#     g_pyvis.show_buttons(filter_=['physics'])

# Note that this does not copy over node and edge attributes, so it's a bit useless really and hence we just rebuild the graph from scratch with pyvis below:
#     g_pyvis.from_nx(g_nx)

#     g_pyvis.add_nodes(
#         [ent['ent'] for ent in ents],
#         color=[ent['color'] for ent in ents],
#         title=[
#             pd.DataFrame(
#                 [(key[1], ent[key[0]])
#                  for key in keys_display_order
#                  if key[0] in ent.keys()],
#             ).style.set_table_styles([
#                 {'selector': 'th.col_heading',
#                  'props': [('display', 'none')]},
#                 {'selector': 'th.row_heading',
#                  'props': [('display', 'none')]},
#             ]).render()
#             for ent in ents
#         ],
#     )

# Temporarily allowing all POS while testing:
    for token in tokens: # [token for token in tokens if (token['pos'] not in {'DET', 'CCONJ', 'PUNCT', 'SYM'})]
        g_pyvis.add_node(
            token['idx_tok'],
            label=str(token['idx_tok']) + ': ' + token['pos'] + ': ' + token['text'],
            title=str(token['idx_tok']) + ': ' + token['pos'] + ': ' + token['text'],
            font='14px gillsans #424242',
            color=ents[token['text']]['color'] if (token['text'] in ents.keys()) else '#EBEBEB',
        )
    for token in tokens:
        g_pyvis.add_edge(
            token['idx_tok_head'],
            token['idx_tok'],
            label=token['dep'],
            title=token['dep'],
            font='12px gillsans-italic #A9A9A9',
        )
    g_pyvis.force_atlas_2based(
        gravity=-100,
        central_gravity=0.005,
        spring_length=50,
        spring_strength=0.1,
        damping=0.5,
        overlap=0.5,
    )
    g_pyvis.show('images/DI_NLP_Graph_Raw.html')
    accordion_graph_raw.selected_index = 0
    display(
        IFrame(
            'images/DI_NLP_Graph_Raw.html',
            width='100%',
            height='1000px',
        ),
    )

@output_graph_con.capture(clear_output=True, wait=True)
def set_output_graph_con(nodes, edges, ents):
    g_pyvis = Network(
        directed=True,
        notebook=True,
        width='100%',
        height='100%',
    )
    g_pyvis.prep_notebook()
    g_pyvis.force_atlas_2based(
        gravity=-100,
        central_gravity=0.005,
        spring_length=50,
        spring_strength=0.1,
        damping=0.5,
        overlap=0.5,
    )
    for node in nodes:
        g_pyvis.add_node(
            node['idx_tok'],
            label=str(node['idx_tok']) + ': ' + node['pos'] + ': ' + node['text'],
            title=str(node['idx_tok']) + ': ' + node['pos'] + ': ' + node['text'],
            font='14px gillsans #424242',
            color=ents[node['text']]['color'] if (node['text'] in ents.keys()) else '#EBEBEB',
        )
    for edge in edges:
        g_pyvis.add_edge(
            edge['idx_tok_from'],
            edge['idx_tok_to'],
            label=edge['di_dep'],
            title=edge['di_dep'],
            font='12px gillsans-italic #A9A9A9',
        )
    g_pyvis.show('images/DI_NLP_Graph_Con.html')
    accordion_graph_con.selected_index = 0
    display(
        IFrame(
            'images/DI_NLP_Graph_Con.html',
            width='100%',
            height='1000px',
        ),
    )

@output_map.capture(clear_output=True, wait=True)
def set_output_map(ents):
    m = Map(
        center=(20.0, 0.0),
        zoom=2,
    )
    markers = []
    for ent in ents.keys():
        if  (ents[ent]['ent_type'] == 'LOC') \
        and ('geo_lat' in ents[ent].keys()) \
        and ('geo_lon' in ents[ent].keys()):
            markers.append(Marker(
                location=(ents[ent]['geo_lat'], ents[ent]['geo_lon']),
                title=ent,
                draggable=False,
            ))
    m.add_layer(LayerGroup(
        layers=tuple(markers),
    ))
#     m.add_layer(MarkerCluster(
#         markers=tuple(markers),
#     ))
    m.add_control(FullScreenControl())
    accordion_map.selected_index = 0
    display(m)

@output_table.capture(clear_output=True, wait=True)
def set_output_table(ents, keys_display_order):
    ents = [ents[ent] for ent in ents.keys()]
    ents = sorted(ents, key=lambda ent:ent['tokens_idx'][0])
    df = pd.DataFrame(
        [[ent[key[0]] if key[0] in ent.keys() else ''
          for key in keys_display_order]
         for ent in ents],
        columns=[key[1] for key in keys_display_order],
    )
    qgrid_widget = qgrid.show_grid(
        df,
        grid_options={
            'enableColumnReorder': True,
        },
#         show_toolbar=True, # This gives a full-screen button, but also add/remove row buttons which we don't want ...
    )
    accordion_table.selected_index = 0
    display(qgrid_widget)
#     df = df.style.set_table_styles([
#         {'selector': 'td',
#          'props': [('text-align', 'left')]},
#         {'selector': 'th.col_heading',
#          'props': [('text-align', 'left')]},
#         {'selector': 'th.row_heading',
#          'props': [('text-align', 'right')]},
#     ])
#     display(df)
#     display(TableDisplay(df))

In [ ]:
def toggle(dummy):
    if (accordion_text.selected_index == 0) \
    or (accordion_tree.selected_index == 0) \
    or (accordion_graph_raw.selected_index == 0) \
    or (accordion_graph_con.selected_index == 0) \
    or (accordion_map.selected_index == 0) \
    or (accordion_table.selected_index == 0):
        accordion_text.selected_index = None
        accordion_tree.selected_index = None
        accordion_graph_raw.selected_index = None
        accordion_graph_con.selected_index = None
        accordion_map.selected_index = None
        accordion_table.selected_index = None
    else:
        accordion_text.selected_index = 0
        accordion_tree.selected_index = 0
        accordion_graph_raw.selected_index = 0
        accordion_graph_con.selected_index = 0
        accordion_map.selected_index = 0
        accordion_table.selected_index = 0

def clear_output(dummy):
    output_text.clear_output()
    output_tree.clear_output()
    output_graph_raw.clear_output()
    output_graph_con.clear_output()
    output_map.clear_output()
    output_table.clear_output()
    accordion_text.selected_index = None
    accordion_tree.selected_index = None
    accordion_graph_raw.selected_index = None
    accordion_graph_con.selected_index = None
    accordion_map.selected_index = None
    accordion_table.selected_index = None

def clear(dummy):
    input_text.value=''
    output_message.clear_output()
    clear_output(None)

def textcat(dummy):
    if   (not input_url.value) and (not input_text.value):
        set_output_message('ERROR: Enter TextCat URL and text for analysis')
    elif (not input_url.value):
        set_output_message('ERROR: Enter TextCat URL')
    elif (not input_text.value):
        set_output_message('ERROR: Enter text for analysis')
    else:
        set_output_message('Processing ...')
# TODO: Explicitly specify as unicode input
        r = requests.post(
            input_url.value,
            data=json.dumps({'text': input_text.value}),
            params={
                'return_corefs_neural': 'true',
                'return_app': 'true',
            },
            headers={
                'Content-Type': 'application/json'
            },
        )
        if r.ok and r.json():
            set_output_message('Done in: ' + str(r.json()['timer']) + ' s')
            set_output_table(r.json()['ents'], r.json()['keys_display_order'])
            set_output_map(r.json()['ents'])
            set_output_graph_con(r.json()['nodes'], r.json()['edges'], r.json()['ents']) #, r.json()['corefs'] if ('corefs' in r.json().keys()) else None)
            set_output_graph_raw(r.json()['tokens'], r.json()['ents'])
#             set_output_tree(r.json()['html_dep'])
            set_output_text(r.json()['html_ent'])
        else:
            set_output_message('ERROR: No data received')
            clear_output(None)

In [ ]:
button_toggle = widgets.Button(
    icon='list',
    tooltip='Toggle all output displays',
    layout={
        'width': '50px',
    },
)
button_toggle.on_click(toggle)

button_clear = widgets.Button(
    icon='refresh',
    tooltip='Clear input and output',
    layout={
        'width': '50px',
    },
)
button_clear.on_click(clear)

button_go = widgets.Button(
    icon='play',
    tooltip='Go!',
    layout={
        'width': '50px',
    },
)
button_go.on_click(textcat)

In [ ]:
VBox_main = widgets.VBox(
    children=[
        image_header,
        input_url,
        input_text,
        widgets.HBox(
            children=[
                button_toggle,
                button_clear,
                button_go,
                output_message,
            ],
        ),
        accordion_text,
#         accordion_tree,
        accordion_graph_raw,
        accordion_graph_con,
        accordion_map,
        accordion_table,
    ],
    layout={
        'width': '90%',
        'left': '10%',
        'padding': '5px',
#         'border': '1px solid black',
    },
)
clear(None)
display(VBox_main)